In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate
import scipy.signal

import pymedphys
import pymedphys._wlutz.pylinac
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findfield
import pymedphys._wlutz.findbb
import pymedphys._wlutz.reporting

In [ ]:
penumbra = 2
edge_lengths = [20, 24]
initial_rotation = 0
bb_diameter = 8

pd.set_option("display.max_rows", 101)

In [ ]:
# session_dir = pathlib.Path(r'S:\Physics\Programming\data\MVISO\RCCC\2619\20191105')
# input_dir = session_dir.joinpath('input')
# output_dir = session_dir.joinpath('output')

frame_paths_list = pymedphys.zip_data_paths("wlutz_arc_session.zip", check_hashes=False)

frame = [path.stem.split('_')[1] for path in frame_paths_list]
timestamps = [path.parent.stem for path in frame_paths_list]
directions = [path.parent.parent.stem for path in frame_paths_list]
beams = [path.parent.parent.parent.stem for path in frame_paths_list]

keys = list(zip(beams, directions, timestamps, frame))


image_paths = {
    key: path for key, path in zip(keys, frame_paths_list)
}

In [ ]:
# output_dir.joinpath(*keys[0])

In [ ]:
keys[0]

In [ ]:
# results_dirs = {
#     key: output_dir.joinpath(*key[0:-1]) for key in keys
# }

# for _, results_dir in results_dirs.items():
#     results_dir.mkdir(exist_ok=True, parents=True)

# field_pos_paths = {
#     key: path.joinpath(f"{key[-1]}_field_pos.csv")
#     for key, path in results_dirs.items()
# }

# bb_pos_paths = {
#     key: path.joinpath(f"{key[-1]}_bb_pos.csv")
#     for key, path in results_dirs.items()
# }

# analysis_img_paths = {
#     key: path.joinpath(f"{key[-1]}_analysis.png")
#     for key, path in results_dirs.items()
# }

# # analysis_img_paths

In [ ]:
for key, image_path in image_paths.items():
    print(key)
    x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)
    
    field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
    initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, img)
    
    try:
        field_centre, field_rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
            field, edge_lengths, penumbra, initial_centre, initial_rotation=initial_rotation
        )
    except ValueError as e:
        print(e)
        continue  
    
        
    field_centre = np.round(field_centre, decimals=2).tolist()
    field_rotation = np.round(field_rotation, decimals=1)

#     pd.DataFrame(
#         data=[[field_centre[0], field_centre[1], field_rotation]], 
#         columns=['x', 'y', 'rotation']).to_csv(field_pos_paths[key], index=False)
    
    try:
        bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
            field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation
        )
    except ValueError as e:
        print(e)
        continue
    
    bb_centre = np.round(bb_centre, decimals=2).tolist()
    
#     pd.DataFrame(
#         data=[bb_centre], 
#         columns=['x', 'y']).to_csv(bb_pos_paths[key], index=False)
    
    fig = pymedphys._wlutz.reporting.image_analysis_figure(
        x,
        y,
        img,
        bb_centre,
        field_centre,
        field_rotation,
        bb_diameter,
        edge_lengths,
        penumbra,
    )
    
    plt.show()
#     fig.savefig(analysis_img_paths[key])
#     plt.close(fig)

In [ ]:
# for key, image_path in image_paths.items():

#     if not field_pos_paths[key].exists() or not bb_pos_paths[key].exists():
#         print(key)
        
#         x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)

#         field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
#         initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, img)

#         try:
#             field_centre, field_rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
#                 field, edge_lengths, penumbra, initial_centre, initial_rotation=initial_rotation
#             )
#         except ValueError as e:
#             print(e)
#             continue

#         field_centre = np.round(field_centre, decimals=2).tolist()
#         field_rotation = np.round(field_rotation, decimals=1)

#         pd.DataFrame(
#             data=[[field_centre[0], field_centre[1], field_rotation]], 
#             columns=['x', 'y', 'rotation']).to_csv(field_pos_paths[key], index=False)
        
#         try:
#             bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
#                 field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation
#             )
#         except ValueError as e:
#             print(e)
#             continue

#         bb_centre = np.round(bb_centre, decimals=2).tolist()

#         pd.DataFrame(
#             data=[bb_centre], 
#             columns=['x', 'y']).to_csv(bb_pos_paths[key], index=False)

#         fig = pymedphys._wlutz.reporting.image_analysis_figure(
#             x,
#             y,
#             img,
#             bb_centre,
#             field_centre,
#             field_rotation,
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#         )

#         fig.savefig(analysis_img_paths[key])
#         plt.close(fig)